In [8]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

#Our helpers
from map_helpers import *
from data_helpers import *
from helpers import *

from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA

import json
import folium
from folium import plugins

#Plot data
import seaborn as sns

%matplotlib inline  

For a better vizualisation of this notebook please read it using the [notebook viewer](https://nbviewer.jupyter.org/github/cgallay/Ada/blob/master/Project/Projet_ADA_M2.ipynb)

# Introduction to the million song Dataset

According to the web site :  https://labrosa.ee.columbia.edu/millionsong "The Million Song Dataset is a freely-available collection of audio features and metadata for a million contemporary popular music tracks.
The core of the dataset is the feature analysis and metadata for one million songs, provided by The Echo Nest. The dataset does not include any audio, only the derived features."

The entire dataset size is 280 GB. We however, based our final analysis on a subset of roughly 100,000 songs. We decided not tu use spark as we could manage a third of the dataset with standard methods and was a sufficiant size for our analysis.

You will see below the first steps of our approach.

### Features description : 
The list of the 54 features present in the dataset, can be found here : https://labrosa.ee.columbia.edu/millionsong/pages/example-track-description
In order not to list them all we choose here to describe the one that seems reelevent to our project: 

 * **artist_name** 
 * **artist latitude:**	float	latitude
 * **artist longitude:** float	longitude
 * **artist_terms:** tags from The Echo Nest in float with range : [0:1]
 * **artist_terms_freq:** similar to mbtags_count but from Echo Nest in float with range : [0:1]  
 * **artist_terms_weight:** same shape as the two previous tags in float with range : [0:1]
 * **loudness:** the overall loudness of a track in decibels (dB). Loudness values in the Analyzer are averaged across an entire track and are useful for comparing relative loudness of segments and tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude).
 * **song_hotttnesss:** according to The Echo Nest, when downloaded (in December 2010) in float with range [0:1]
 * **segments_pitches** The key is a track-level attribute with range : [0:11] and corresponding to one of the 12 keys: C, C#, D, etc. up to B. If no key was detected, the value is -1. The mode is equal to 0 or 1 for “minor” or “major” and may be -1 in case of no result.
 * **segments_timbre** timbre is the quality of a musical note or sound that distinguishes different types of musical instruments, or voices. It is a complex notion also referred to as sound color, texture, or tone quality, and is derived from the shape of a segment’s spectro-temporal surface, independently of pitch and loudness.
 * **tempo:** tempo in BPM according to The Echo Nest
 * **title**
 * **year:** when this song was released, according to musicbrainz.org

# Importing Data
We proceded in two steps:

- Firtly, we loaded the dataset by batches and, for each of them, we extracted the useful features. We saved all the data in pickle formats for faster loading.

- Secondly, we loaded the corresponding pickles according to our needs.

In [ ]:
#Generating all pickles (WARNING: Takes ~2 Hours)
for letter in ['A', 'B', 'C', 'D', 'E', 'F']:
    for half in [1, 2]:
        df = load_song_data(letter, half)
        df1 = select_col(filter_year(df), part=1).dropna()
        df2 = select_col(exctract_timbre_features(filter_hotness(df, 0.001)), part=2).dropna()
        save_pickle_filtered(df1, letter, part=1, half=half)
        save_pickle_filtered(df2, letter, part=2, half=half)
        del df, df1, df2

## Part 1: Discovering Genre through time

In [12]:
# loading the corresponding subdataset for the first part
df_song = merge_pickles(['A', 'B', 'C', 'D', 'E', 'F'], 1)

In [10]:
df_song

,artist_latitude,artist_longitude,artist_terms,artist_terms_weight,year
3,40.71455,-74.00712,"[glam metal, power metal, heavy metal, progres...","[1.0, 0.96293777998, 0.95456651193, 0.94632947...",2005
7,35.22250,-80.83754,"[rock, urban, rap, romantic, vocal, classic, s...","[1.0, 0.949062952363, 0.706145757692, 0.678399...",2000
10,40.85251,-73.13585,"[alternative, new york, grunge, classic, power...","[1.0, 0.999999570835, 0.71283464117, 0.6974758...",2001
12,42.36679,-71.10602,"[rock, jazz, soundtrack, blues, pop, ambient, ...","[1.0, 0.967475163519, 0.875256369098, 0.872488...",2000
22,35.22250,-80.83754,"[alternative metal, hardcore punk, progressive...","[1.0, 0.954614238333, 0.94968209112, 0.8616207...",2002
25,33.74831,-84.39111,"[salsa, pop, latin, folk, tropical, merengue, ...","[1.0, 0.765895496001, 0.730657319557, 0.730649...",1990
27,34.05349,-118.24532,"[rap metal, post-grunge, nu metal, rapcore, al...","[1.0, 1.0, 0.994887580772, 0.991536207153, 0.9...",2000
28,30.26760,-97.74298,"[honky tonk, western swing, roots rock, ragtim...","[1.0, 0.999126510757, 0.995462870586, 0.966130...",1999
29,43.04181,-87.90684,"[soul jazz, jazz funk, cool jazz, swing, big b...","[1.0, 0.975384096228, 0.933196599769, 0.920601...",1988
39,35.32689,-83.80489,"[country rock, southern rock, classic country,...","[1.0, 0.873692817335, 0.867459449486, 0.857020...",1980
